In [83]:
import yaml

In [84]:
from dask import dataframe as dd
df = dd.read_csv('C:/Users/HAZARIKA/Desktop/Rate.csv',delimiter=',')

In [85]:
df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 24 entries, BusinessYear to RowNumber
dtypes: object(10), float64(9), int64(5)

In [87]:
#No. of Rows
len(df.index)

12694445

In [19]:
# remove special character
df.columns=df.columns.str.replace('[#,@,&]','')

<ipython-input-19-554f595a0ef5>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns=df.columns.str.replace('[#,@,&]','')


In [20]:
#To remove white space from columns
df.columns = df.columns.str.replace(' ', '')

In [21]:
data=df.columns
data

Index(['BusinessYear', 'StateCode', 'IssuerId', 'SourceName', 'VersionNum',
       'ImportDate', 'IssuerId2', 'FederalTIN', 'RateEffectiveDate',
       'RateExpirationDate', 'PlanId', 'RatingAreaId', 'Tobacco', 'Age',
       'IndividualRate', 'IndividualTobaccoRate', 'Couple',
       'PrimarySubscriberAndOneDependent', 'PrimarySubscriberAndTwoDependents',
       'PrimarySubscriberAndThreeOrMoreDependents', 'CoupleAndOneDependent',
       'CoupleAndTwoDependents', 'CoupleAndThreeOrMoreDependents',
       'RowNumber'],
      dtype='object')

In [22]:
#Write Python objects to YAML file

In [23]:
with open('C:/Users/HAZARIKA/Desktop/store.yaml', 'w') as f:
    yaml.dump(data, f, default_flow_style=False)

In [24]:
with open('C:/Users/HAZARIKA/Desktop/store.yaml') as f:
    print(f.read())

!!python/object/apply:pandas.core.indexes.base._new_Index
- !!python/name:pandas.core.indexes.base.Index ''
- data: !!python/object/apply:numpy.core.multiarray._reconstruct
    args:
    - !!python/name:numpy.ndarray ''
    - !!python/tuple
      - 0
    - !!binary |
      Yg==
    state: !!python/tuple
    - 1
    - !!python/tuple
      - 24
    - !!python/object/apply:numpy.dtype
      args:
      - O8
      - false
      - true
      state: !!python/tuple
      - 3
      - '|'
      - null
      - null
      - null
      - -1
      - -1
      - 63
    - false
    - - BusinessYear
      - StateCode
      - IssuerId
      - SourceName
      - VersionNum
      - ImportDate
      - IssuerId2
      - FederalTIN
      - RateEffectiveDate
      - RateExpirationDate
      - PlanId
      - RatingAreaId
      - Tobacco
      - Age
      - IndividualRate
      - IndividualTobaccoRate
      - Couple
      - PrimarySubscriberAndOneDependent
      - PrimarySubscriberAndTwoDependents
      - Prima

In [25]:
#Reading from YAML file
import yaml

with open('C:/Users/HAZARIKA/Desktop/store.yaml', 'r') as f:
    column_list=yaml.load(f, yaml.Loader)
    print(column_list)

Index(['BusinessYear', 'StateCode', 'IssuerId', 'SourceName', 'VersionNum',
       'ImportDate', 'IssuerId2', 'FederalTIN', 'RateEffectiveDate',
       'RateExpirationDate', 'PlanId', 'RatingAreaId', 'Tobacco', 'Age',
       'IndividualRate', 'IndividualTobaccoRate', 'Couple',
       'PrimarySubscriberAndOneDependent', 'PrimarySubscriberAndTwoDependents',
       'PrimarySubscriberAndThreeOrMoreDependents', 'CoupleAndOneDependent',
       'CoupleAndTwoDependents', 'CoupleAndThreeOrMoreDependents',
       'RowNumber'],
      dtype='object')


In [ ]:
#Validation

In [34]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re


def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.load(stream, Loader=yaml.Loader)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Overwriting testutility.py


In [72]:
%%writefile file.yaml
file_type: csv
dataset_name: testfile
file_name: test_data
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - BusinessYear
      - StateCode
      - IssuerId
      - SourceName
      - VersionNum
      - ImportDate

Writing file.yaml


In [73]:
# Read config file
import testutility as util
config_data = util.read_config_file("file.yaml")

In [74]:
#inspecting data of config file
config_data

{'file_type': 'csv',
 'dataset_name': 'testfile',
 'file_name': 'test_data',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['BusinessYear - StateCode - IssuerId - SourceName - VersionNum - ImportDate']}

In [75]:
# Normal reading process of the file
from dask import dataframe as dd
df_sample = dd.read_csv('C:/Users/HAZARIKA/Desktop/Rate.csv',delimiter=',')
df_sample.head()

,BusinessYear,StateCode,IssuerId,SourceName,VersionNum,ImportDate,IssuerId2,FederalTIN,RateEffectiveDate,RateExpirationDate,...,IndividualRate,IndividualTobaccoRate,Couple,PrimarySubscriberAndOneDependent,PrimarySubscriberAndTwoDependents,PrimarySubscriberAndThreeOrMoreDependents,CoupleAndOneDependent,CoupleAndTwoDependents,CoupleAndThreeOrMoreDependents,RowNumber
0,2014,AK,21989,HIOS,6,2014-03-19 07:06:49,21989,93-0438772,2014-01-01,2014-12-31,...,29.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14
1,2014,AK,21989,HIOS,6,2014-03-19 07:06:49,21989,93-0438772,2014-01-01,2014-12-31,...,36.95,NaN,73.9,107.61,107.61,107.61,144.56,144.56,144.56,14
2,2014,AK,21989,HIOS,6,2014-03-19 07:06:49,21989,93-0438772,2014-01-01,2014-12-31,...,36.95,NaN,73.9,107.61,107.61,107.61,144.56,144.56,144.56,15
3,2014,AK,21989,HIOS,6,2014-03-19 07:06:49,21989,93-0438772,2014-01-01,2014-12-31,...,32.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15
4,2014,AK,21989,HIOS,6,2014-03-19 07:06:49,21989,93-0438772,2014-01-01,2014-12-31,...,32.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16


In [76]:
# read the file using config file
file_type = config_data['file_type']
source_file = config_data['file_name'] + f'.{file_type}'

In [77]:
#print("",source_file)
df = pd.read_csv(source_file,config_data['inbound_delimiter'])
df.head()

,BusinessYear,StateCode,IssuerId,SourceName,VersionNum,ImportDate,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,2014,AK,21989,HIOS,6,19/03/2014 07:06,NaN,NaN,NaN,NaN
1,2014,AK,21989,HIOS,6,19/03/2014 07:06,NaN,NaN,NaN,NaN
2,2014,AK,21989,HIOS,6,19/03/2014 07:06,NaN,NaN,NaN,NaN
3,2014,AK,21989,HIOS,6,19/03/2014 07:06,NaN,NaN,NaN,NaN
4,2014,AK,21989,HIOS,6,19/03/2014 07:06,NaN,NaN,NaN,NaN


In [78]:
#validate the header of the file
util.col_header_val(df,config_data)

column name and column length validation failed
Following File columns are not in the YAML file ['unnamed_7', 'statecode', 'sourcename', 'importdate', 'unnamed_8', 'unnamed_9', 'businessyear', 'versionnum', 'issuerid', 'unnamed_6']
Following YAML columns are not in the file uploaded ['businessyear - statecode - issuerid - sourcename - versionnum - importdate']


0

In [79]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['businessyear', 'statecode', 'issuerid', 'sourcename', 'versionnum',
       'importdate', 'unnamed_6', 'unnamed_7', 'unnamed_8', 'unnamed_9'],
      dtype='object')
columns of YAML are: ['BusinessYear - StateCode - IssuerId - SourceName - VersionNum - ImportDate']


In [80]:
if util.col_header_val(df,config_data)==0:
    print("validation failed")
else:
    print("col validation passed")

column name and column length validation failed
Following File columns are not in the YAML file ['unnamed_7', 'statecode', 'sourcename', 'importdate', 'unnamed_8', 'unnamed_9', 'businessyear', 'versionnum', 'issuerid', 'unnamed_6']
Following YAML columns are not in the file uploaded ['businessyear - statecode - issuerid - sourcename - versionnum - importdate']
validation failed
